<a href="https://colab.research.google.com/github/Skander28/Models/blob/main/Cnn_bilstm_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/

 Classroom	    dialect_dataset.csv   preprocessed_tweets.csv
'Colab Notebooks'   messages.csv


In [ ]:
import pandas as pd
filtered_df = pd.read_csv('/content/drive/MyDrive/preprocessed_tweets.csv')
filtered_df.head()

,Unnamed: 0,id,dialect,tweets
0,15497,1009754958479151232,1,قليلين ادب ومنافقين اختهم او قريبتهم تعاكس تقو...
1,15498,1009794751548313600,1,اليبين متقلبين بالنسبه ليا انا ميليشياوي زمان ...
2,15499,1019989115490787200,1,تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ انها ...
3,15500,1035479791758135168,1,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل...
4,15501,1035481122921164800,1,شكلك متعقده علشان الراجل تحبيه ازوج بنت يتيمه ...


In [ ]:
filtered_df = filtered_df.dropna()

In [ ]:
# Load data
features = filtered_df.tweets.values
labels = pd.get_dummies(filtered_df['dialect']).values

In [ ]:
vocab_size = 20000
max_length= 200
tokenizer = Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(features)
X = tokenizer.texts_to_sequences(features)

In [ ]:
import nltk
from collections import Counter
nltk.download('stopwords')
from nltk.corpus import stopwords

# use the stopwords
stop_words = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
all_words = []
for tokens in X:
    all_words.extend(tokens)

stop_words = set(stopwords.words('arabic'))
all_words = [word for word in all_words if word not in stop_words]

word_counts = Counter(all_words)
most_common_words = [word for word, count in word_counts.most_common(500)]

def remove_common_words(tokens):
  new_tokens = [token for token in tokens if token not in most_common_words]
  return new_tokens

X = remove_common_words(X)


X = pad_sequences(X, maxlen=max_length)

In [ ]:
# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus=RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train,y_train)

In [ ]:
# Convert data to PyTorch tensors
X_res_, y_res_ = torch.tensor(X_res), torch.tensor(y_res)
X_val, y_val = torch.tensor(X_val), torch.tensor(y_val)
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

In [ ]:
class DialectDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# Create dataloaders for training, validation, and test sets
train_dataset = DialectDataset(X_res, y_res)
val_dataset = DialectDataset(X_val, y_val)
test_dataset = DialectDataset(X_test, y_test)

In [ ]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np

vocab_size = 20000
embedding_dim = 100
hidden_dim = 128
output_dim = 4
num_layers = 2
bidirectional = True
lr = 0.01
batch_size = 64
num_epochs = 100
patience = 5

class BiLSTMAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers):
        super(BiLSTMAttention, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, embedding_dim), padding=(1, 0))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 1), padding=(1, 0))
        self.lstm = nn.LSTM(200, hidden_dim, num_layers=num_layers, bidirectional=True, dropout=0.5, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(0.5)
        self.attention = SelfAttention(hidden_dim * 2)

    def forward(self, x):
        embedded = self.embedding(x)
        conv1_out = F.relu(self.conv1(embedded.unsqueeze(1)))
        conv2_out = F.relu(self.conv2(conv1_out))
        conv2_out = conv2_out.permute(0, 2, 1, 3)
        conv2_out = conv2_out.reshape(conv2_out.size(0), conv2_out.size(1), -1)
        lstm_in = conv2_out.permute(0, 2, 1)
        lstm_out, _ = self.lstm(lstm_in)
        lstm_out = self.dropout(lstm_out)
        attention_out , attention_weights= self.attention(lstm_out)
        fc_out = self.fc(attention_out)
        return fc_out, attention_weights

class SelfAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, encoder_outputs):
        energy = self.projection(encoder_outputs)
        weights = F.softmax(energy.flatten(start_dim=1), dim=1)
        outputs = (encoder_outputs * weights.unsqueeze(-1)).sum(dim=1)
        return outputs, weights

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BiLSTMAttention(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers).to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01)
criterion = nn.BCEWithLogitsLoss()
counter_nb = 0
for epoch in tqdm(range(24)):
     model.train()
     counter_nb = counter_nb + 1 
     print(counter_nb)
     running_loss = 0.0
     with torch.cuda.device(0):
       for batch in train_dataloader:
         inputs, labels = batch[0].to(device), batch[1].to(device)
         optimizer.zero_grad()
         labels = labels.float()
         outputs, _ = model(inputs)
         loss = criterion(outputs, labels)
         loss.backward()
         optimizer.step()
         running_loss += loss.item()
         preds = torch.sigmoid(outputs) > 0.5
         acc = accuracy_score(labels.cpu().detach().numpy(), preds.cpu().detach().numpy())
         #print('Train Accuracy: {:.4f}'.format(acc))
     epoch_loss = running_loss / (len(train_dataloader))

     best_val_loss = np.inf
     patience = 3
     counter = 0
     val_loss = 0.0
     model.eval()
     with torch.no_grad():
         for batch in val_dataloader:
             inputs, labels = batch[0].to(device), batch[1].to(device)
             labels = labels.float()
             outputs, _= model(inputs)
             loss = criterion(outputs, labels)
             val_loss += loss.item()
             val_preds = torch.sigmoid(outputs) > 0.5
             val_acc = accuracy_score(labels.cpu().detach().numpy(), val_preds.cpu().detach().numpy())
             #print('Val Accuracy: {:.4f}'.format(val_acc))
     val_loss = val_loss / (len(val_dataloader))
     # Check if the validation loss has improved
     if val_loss < best_val_loss:
                best_val_loss = val_loss
                counter = 0
     else:
         counter += 1

     # Stop the training process if the validation loss hasn't improved for `patience` epochs
     if counter >= patience:
         break
     print(" epoch loss :", epoch_loss , "| val loss :", val_loss)
     print('Train Accuracy: {:.4f}'.format(acc))
     print('Val Accuracy: {:.4f}'.format(val_acc))

print("Training stopped after epoch", epoch)
        

  0%|          | 0/24 [00:00<?, ?it/s]

1


  4%|▍         | 1/24 [00:17<06:39, 17.36s/it]

 epoch loss : 0.515369971400948 | val loss : 0.4149530813671075
Train Accuracy: 0.5227
Val Accuracy: 0.5294
2


  8%|▊         | 2/24 [00:34<06:16, 17.13s/it]

...

 epoch loss : 0.08450504604286405 | val loss : 0.7288854924532083
Train Accuracy: 0.8409
Val Accuracy: 0.7647
23


 96%|█████████▌| 23/24 [06:30<00:16, 16.94s/it]

 epoch loss : 0.08543788136974458 | val loss : 0.8208388491318777
Train Accuracy: 0.9318
Val Accuracy: 0.7647
24


100%|██████████| 24/24 [06:47<00:00, 16.98s/it]

 epoch loss : 0.08103359503451214 | val loss : 0.7549706095686326
Train Accuracy: 0.9091
Val Accuracy: 0.8235
Training stopped after epoch 23


In [ ]:
from sklearn.metrics import f1_score
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for batch in test_dataloader:
        inputs, labels = batch[0].to(device), batch[1].to(device)
        labels = labels.float()
        outputs, _ = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        y_true.extend(labels.cpu().detach().numpy())
        y_pred.extend(preds.cpu().detach().numpy())
test_acc = accuracy_score(y_true, y_pred)
test_f1 = f1_score(y_true, y_pred, average='macro')
print('Test Accuracy: {:.4f}'.format(test_acc))

Test Accuracy: 0.6388


In [ ]:
torch.save(model.state_dict(), 'model2.pth')